# MAML Notebook

The training process of MAML

## 1. Load libraries


In [ ]:
%cd '/content/drive/MyDrive/Meta_learning_research/Notebooks/'
%pip install wandb
import os
import wandb
import numpy as np
import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
from data_util import MetaDataLoader

/content/drive/MyDrive/Meta_learning_research/Notebooks


## 2. Prepare training data

Each episode is one location (task).

In [ ]:
data_dir = './samples/'  # Replace with the path to your directory containing numpy files
locations_meta_training = ['Alexander', 'Rowancreek']
locations_meta_testing = ['Covington']
num_samples_per_location = 25  # Configure the number of samples per location
num_episodes = 10  # Number of episodes
normalization_type='-1' # the lower end of the normalized range  ("0" or "-1")
data_loader = MetaDataLoader(data_dir, num_samples_per_location, normalization_type)

In [ ]:
# Create multi episodes for meta-training
meta_train_episodes = data_loader.create_multi_episodes(num_episodes, locations_meta_training)

Alexander
Alexander
Rowancreek
Rowancreek
Alexander
Rowancreek
Alexander
Rowancreek
Rowancreek
Alexander


## 3. Define the model

We are using simple U-net model with 700K trainable variables.  

### Define Dice_loss (same as F1-score)

In [ ]:
# Dice coefficient (similar to F1 score but differentiable)
def dice_coefficient(y_true, y_pred):
    smooth = 1e-6  # Small constant to avoid division by zero
    y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
    y_pred_f = tf.cast(tf.reshape(y_pred, [-1]), tf.float32)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

# Dice loss to be minimized
def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

### Define Attention U-net model

In [ ]:
from tensorflow.keras.layers import ( Input, Conv2D, BatchNormalization, Activation, Conv2DTranspose,
                                        MaxPooling2D, Layer, add, multiply, GlobalAveragePooling2D,
                                        Dense, Reshape, Multiply )
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

class ChannelAttention(Layer):
    def __init__(self, reduction_ratio=8, **kwargs):
        super(ChannelAttention, self).__init__(**kwargs)
        self.reduction_ratio = reduction_ratio

    def build(self, input_shape):
        channel = input_shape[-1]
        self.fc1 = Dense(channel // self.reduction_ratio, activation='relu', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')
        self.fc2 = Dense(channel, activation='sigmoid', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')
        super(ChannelAttention, self).build(input_shape)

    def call(self, inputs):
        avg_pool = GlobalAveragePooling2D()(inputs)
        avg_pool = Reshape((1, 1, avg_pool.shape[1]))(avg_pool)
        fc1_out = self.fc1(avg_pool)
        fc2_out = self.fc2(fc1_out)
        scale = Multiply()([inputs, fc2_out])
        return scale

    def compute_output_shape(self, input_shape):
        return input_shape

    def get_config(self):
        config = super(ChannelAttention, self).get_config()
        config.update({'reduction_ratio': self.reduction_ratio})
        return config

class SpatialAttention(Layer):
    def __init__(self, kernel_size=7, **kwargs):
        super(SpatialAttention, self).__init__(**kwargs)
        self.kernel_size = kernel_size

    def build(self, input_shape):
        self.conv = Conv2D(1, self.kernel_size, padding='same', activation='sigmoid', kernel_initializer='he_normal', use_bias=False)
        super(SpatialAttention, self).build(input_shape)

    def call(self, inputs):
        avg_pool = K.mean(inputs, axis=3, keepdims=True)
        max_pool = K.max(inputs, axis=3, keepdims=True)
        concat = K.concatenate([avg_pool, max_pool], axis=3)
        attention = self.conv(concat)
        return multiply([inputs, attention])

    def compute_output_shape(self, input_shape):
        return input_shape

    def get_config(self):
        config = super(SpatialAttention, self).get_config()
        config.update({'kernel_size': self.kernel_size})
        return config

class AttentionUnet:
    def __init__(self, img_width=224, input_channels=8, output_mask_channels=1, filters=32, last_dropout=0.2):
        self.img_width = img_width
        self.input_channels = input_channels
        self.output_mask_channels = output_mask_channels
        self.filters = filters
        self.last_dropout = last_dropout

    def residual_cnn_block(self, x, size, dropout=0.0, batch_norm=True):
        conv = Conv2D(size, (3, 3), padding='same')(x)
        if batch_norm:
            conv = BatchNormalization()(conv)
        conv = Activation('relu')(conv)
        conv = Conv2D(size, (3, 3), padding='same')(conv)
        if batch_norm:
            conv = BatchNormalization()(conv)
        conv = Activation('relu')(conv)
        return conv

    def attention_up_and_concatenate(self, inputs, attention_type):
        g, x = inputs
        if attention_type == 'spatial':
            attention_layer = SpatialAttention()
        elif attention_type == 'channel':
            attention_layer = ChannelAttention()
        x = attention_layer(x)
        inter_channel = x.get_shape().as_list()[3]
        g = Conv2DTranspose(inter_channel, (3,3), strides=(2, 2), padding='same')(g)
        return add([g, x])

    def build_model(self):
        inputs = Input((self.img_width, self.img_width, self.input_channels))
        filters = self.filters

        conv_224 = self.residual_cnn_block(inputs, filters)
        pool_112 = MaxPooling2D(pool_size=(2, 2))(conv_224)
        conv_112 = self.residual_cnn_block(pool_112, filters * 2)
        pool_56 = MaxPooling2D(pool_size=(2, 2))(conv_112)
        conv_56 = self.residual_cnn_block(pool_56, filters * 4)
        pool_28 = MaxPooling2D(pool_size=(2, 2))(conv_56)
        conv_28 = self.residual_cnn_block(pool_28, filters * 8)
        pool_14 = MaxPooling2D(pool_size=(2, 2))(conv_28)
        conv_14 = self.residual_cnn_block(pool_14, filters * 16)
        pool_7 = MaxPooling2D(pool_size=(2, 2))(conv_14)
        conv_7 = self.residual_cnn_block(pool_7, filters * 32)

        # Upsampling path
        up_14 = self.attention_up_and_concatenate([conv_7, conv_14], 'spatial')
        up_conv_14 = self.residual_cnn_block(up_14, filters * 16)
        up_28 = self.attention_up_and_concatenate([up_conv_14, conv_28], 'spatial')
        up_conv_28 = self.residual_cnn_block(up_28, filters * 8)
        up_56 = self.attention_up_and_concatenate([up_conv_28, conv_56], 'channel')
        up_conv_56 = self.residual_cnn_block(up_56, filters * 4)
        up_112 = self.attention_up_and_concatenate([up_conv_56, conv_112], 'channel')
        up_conv_112 = self.residual_cnn_block(up_112, filters * 2)
        up_224 = self.attention_up_and_concatenate([up_conv_112, conv_224], 'channel')
        up_conv_224 = self.residual_cnn_block(up_224, filters, dropout=self.last_dropout)

        # Output layer
        conv_final = Conv2D(self.output_mask_channels, (1, 1), activation='sigmoid')(up_conv_224)

        # Create model
        model = Model(inputs, conv_final, name="AttentionUnet")
        return model


### Define Simple U-net model

In [ ]:
import tensorflow as tf

class SimpleUNet:
    def __init__(self, input_shape=(224, 224, 8), num_classes=1):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build_model(self):
        inputs = tf.keras.Input(shape=self.input_shape)

        # Downsample
        c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
        c1 = tf.keras.layers.Dropout(0.1)(c1)
        c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
        p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

        # Bottleneck
        c5 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
        c5 = tf.keras.layers.Dropout(0.2)(c5)
        c5 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

        # Upsample
        u6 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c5)
        u6 = tf.keras.layers.concatenate([u6, c1])
        c6 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
        c6 = tf.keras.layers.Dropout(0.1)(c6)
        c6 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

        # Output layer
        outputs = tf.keras.layers.Conv2D(self.num_classes, (1, 1), activation='sigmoid')(c6)

        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        return model

## 4. Define the training process (MAML)

### The training with constant LR


In [ ]:

def train_task_model(model, inputs, outputs, learning_rate=0.001, momentum=0.5):
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum)
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = dice_loss(outputs, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return model, loss  # Return the updated model

def maml_model(base_model, episodes, meta_lr=0.001, inner_lr=0.0035, momentum=0.9, meta_batch_size=1, inner_steps=10, epochs=10):
    meta_optimizer = tf.keras.optimizers.Adam(learning_rate=meta_lr)

    for epoch in range(epochs):
        task_losses = []
        for batch_index in range(meta_batch_size):
            task_updates = []
            for episode_index, episode in enumerate(episodes):
                # Copy model for task-specific training
                model_copy = tf.keras.models.clone_model(base_model)
                model_copy.set_weights(base_model.get_weights())

                print(f"Epoch {epoch + 1}, Meta-batch {batch_index + 1}, Starting training on episode {episode_index + 1}")

                # Inner loop: Task-specific adjustments with momentum using support set
                support_data = episode["support_set_data"]
                support_labels = episode["support_set_labels"]
                for step in range(inner_steps):
                    model_copy, inner_loss= train_task_model(model_copy, support_data, support_labels, learning_rate=inner_lr, momentum=momentum)
                    print(f" -- Inner step {step + 1}, Loss {inner_loss}")

                # Evaluate the adapted model on the query set
                query_data = episode["query_set_data"]
                query_labels = episode["query_set_labels"]
                val_predictions = model_copy(query_data)
                val_loss = dice_loss(query_labels, val_predictions)
                task_losses.append(val_loss.numpy())  # Record validation loss
                print(f" -- Validation loss after adapting to episode {episode_index + 1}: {val_loss.numpy()}")

              # Compute gradients for meta-update using the base model's variables
                with tf.GradientTape() as meta_tape:
                    # Watch the copy's variables directly for changes
                    meta_tape.watch(model_copy.trainable_variables)
                    new_val_loss = dice_loss(query_labels, model_copy(query_data))
                gradients = meta_tape.gradient(new_val_loss, model_copy.trainable_variables)

                # Map gradients back to the base model's variables
                mapped_gradients = [tf.identity(grad) for grad in gradients]
                task_updates.append((mapped_gradients, new_val_loss))

            # Outer loop: Update the base model using aggregated gradients from all tasks
            if task_updates:
                num_variables = len(base_model.trainable_variables)
                mean_gradients = []
                for i in range(num_variables):
                    grads = [update[0][i] for update in task_updates if update[0][i] is not None]
                    if grads:
                        mean_grad = tf.reduce_mean(tf.stack(grads), axis=0)
                        mean_gradients.append(mean_grad)
                    else:
                        mean_gradients.append(None)  # Handle the case where all gradients for a variable are None

                # Only apply gradients that are not None
                gradients_to_apply = [(grad, var) for grad, var in zip(mean_gradients, base_model.trainable_variables) if grad is not None]
                if gradients_to_apply:
                    meta_optimizer.apply_gradients(gradients_to_apply)

        print(f"Epoch {epoch + 1} completed, Mean Validation Loss across all episodes: {tf.reduce_mean(task_losses)}")

    return base_model

### The training with dynamics LR



In [ ]:
def train_task_model(model, inputs, outputs, optimizer):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = dice_loss(outputs, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return model, loss.numpy()

def maml_model(base_model, episodes, initial_meta_lr=0.001, initial_inner_lr=0.001, decay_steps=1000, decay_rate=0.96, meta_batch_size=1, inner_steps=1, epochs=500, patience=15, save_path='models'):

    date_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    name = f'maml_{inner_steps}_{epochs}_{meta_batch_size}_{date_time}_best_model'
    model_path = os.path.join(save_path, name)
    print(f'Initialize the Training process: {name}')

    # Initialize WandB
    wandb.init(project="maml_experiment",
               name = name,
               config={
                    "initial_meta_lr": initial_meta_lr,
                    "initial_inner_lr": initial_inner_lr,
                    "decay_steps": decay_steps,
                    "decay_rate": decay_rate,
                    "meta_batch_size": meta_batch_size,
                    "inner_steps": inner_steps,
                    "epochs": epochs,
                    "patience": patience,
                    "model_path": model_path,
                    "model_type": "unet"
                })


    # Define learning rate schedules
    inner_lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_inner_lr, decay_steps, decay_rate, staircase=True)
    meta_lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_meta_lr, decay_steps, decay_rate, staircase=True)
    meta_optimizer = tf.keras.optimizers.Adam(learning_rate=meta_lr_schedule)

    best_loss = float('inf')
    no_improvement_count = 0  # Counter to track the number of epochs without improvement

    for epoch in range(epochs):
        task_losses = []
        for batch_index in range(meta_batch_size):
            task_updates = []
            for episode_index, episode in enumerate(episodes):
                # Copy model for task-specific training
                model_copy = tf.keras.models.clone_model(base_model)
                model_copy.set_weights(base_model.get_weights())

                inner_optimizer = tf.keras.optimizers.SGD(learning_rate=inner_lr_schedule(epoch * len(episodes) + episode_index))

                # Inner loop: Task-specific adjustments with dynamic learning rate
                support_data = episode["support_set_data"]
                support_labels = episode["support_set_labels"]
                episode_losses = []
                for step in range(inner_steps):
                    model_copy, loss = train_task_model(model_copy, support_data, support_labels, inner_optimizer)
                    episode_losses.append(loss)

                # Evaluate the adapted model on the query set
                query_data = episode["query_set_data"]
                query_labels = episode["query_set_labels"]
                val_predictions = model_copy(query_data)
                val_loss = dice_loss(query_labels, val_predictions)
                task_losses.append(val_loss.numpy())

                wandb.log({
                    "epoch": epoch,
                    "episode": episode_index,
                    "eps_loss": tf.reduce_mean(episode_losses),
                    "eps_val_loss": val_loss.numpy()
                })

                # Compute gradients for meta-update using the base model's variables
                with tf.GradientTape() as meta_tape:
                    meta_tape.watch(model_copy.trainable_variables)
                    new_val_loss = dice_loss(query_labels, model_copy(query_data))
                gradients = meta_tape.gradient(new_val_loss, model_copy.trainable_variables)

                # Map gradients back to the base model's variables
                mapped_gradients = [tf.identity(grad) for grad in gradients]
                task_updates.append((mapped_gradients, new_val_loss))

            # Outer loop: Update the base model using aggregated gradients from all tasks
            if task_updates:
                num_variables = len(base_model.trainable_variables)
                mean_gradients = []
                for i in range(num_variables):
                    grads = [update[0][i] for update in task_updates if update[0][i] is not None]
                    if grads:
                        mean_grad = tf.reduce_mean(tf.stack(grads), axis=0)
                        mean_gradients.append(mean_grad)
                    else:
                        mean_gradients.append(None)  # Handle the case where all gradients for a variable are None

                gradients_to_apply = [(grad, var) for grad, var in zip(mean_gradients, base_model.trainable_variables) if grad is not None]
                if gradients_to_apply:
                    meta_optimizer.apply_gradients(gradients_to_apply)

        mean_loss = tf.reduce_mean(task_losses)
        wandb.log({
            "epoch": epoch,
            "mean_val_loss": mean_loss
        })

        # Early stopping and model saving
        if mean_loss < best_loss:
            best_loss = mean_loss
            no_improvement_count = 0
            base_model.save(model_path)  # Save the best model
            print(f"Saved new best model with validation loss: {best_loss}")

        else:
            no_improvement_count += 1
            if no_improvement_count >= patience:
                print(f"No improvement for {patience} consecutive epochs, stopping training.")
                break  # Stop training if no improvement in 'patience' number of epochs

        print(f"Epoch {epoch + 1} completed, Mean Validation Loss across all episodes: {mean_loss}")

    wandb.finish()
    return base_model, name

## 5. Train the model with the Meta-training set.

In [ ]:
# Example usage:
unet_model = SimpleUNet(input_shape=(224, 224, 8))
model = unet_model.build_model()
model.summary()

# Dynamic LR
adapted_model, name = maml_model(model, meta_train_episodes, initial_meta_lr=0.001, initial_inner_lr=0.001, decay_steps=1000, decay_rate=0.96, meta_batch_size=1, inner_steps=1, epochs=500)
print("model name:", name)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 8)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 224, 224, 16)         1168      ['input_1[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, 224, 224, 16)         0         ['conv2d[0][0]']              
                                                                                                  
 conv2d_1 (Conv2D)           (None, 224, 224, 16)         2320      ['dropout[0][0]']             
                                                                                              

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

Saved new best model with validation loss: 0.17019779980182648
Epoch 180 completed, Mean Validation Loss across all episodes: 0.17019779980182648
Epoch 181 completed, Mean Validation Loss across all episodes: 0.17554374039173126
Epoch 182 completed, Mean Validation Loss across all episodes: 0.18127447366714478
Epoch 183 completed, Mean Validation Loss across all episodes: 0.17577557265758514
Epoch 184 completed, Mean Validation Loss across all episodes: 0.17023026943206787
Epoch 185 completed, Mean Validation Loss across all episodes: 0.18100816011428833
Epoch 186 completed, Mean Validation Loss across all episodes: 0.17251701653003693


Saved new best model with validation loss: 0.1696205586194992
Epoch 187 completed, Mean Validation Loss across all episodes: 0.1696205586194992
Epoch 188 completed, Mean Validation Loss across all episodes: 0.17315752804279327
Epoch 189 completed, Mean Validation Loss across all episodes: 0.17115117609500885


Saved new best model with validation loss: 0.166734978556633
Epoch 190 completed, Mean Validation Loss across all episodes: 0.166734978556633
Epoch 191 completed, Mean Validation Loss across all episodes: 0.16799786686897278
Epoch 192 completed, Mean Validation Loss across all episodes: 0.170289546251297


Saved new best model with validation loss: 0.16659432649612427
Epoch 193 completed, Mean Validation Loss across all episodes: 0.16659432649612427


Saved new best model with validation loss: 0.16490554809570312
Epoch 194 completed, Mean Validation Loss across all episodes: 0.16490554809570312
Epoch 195 completed, Mean Validation Loss across all episodes: 0.1654222309589386
Epoch 196 completed, Mean Validation Loss across all episodes: 0.16638043522834778
Epoch 197 completed, Mean Validation Loss across all episodes: 0.16589495539665222


## 6. Adapt the model to tartget area
After we ge the model that is trained on Alexander and Rowancreek using MAML.  

In [ ]:
def adapt_to_new_task(base_model, support_data, support_labels, inner_lr=0.001, inner_steps=1):
    model_copy = tf.keras.models.clone_model(base_model)
    model_copy.set_weights(base_model.get_weights())

    optimizer = tf.keras.optimizers.Adam(learning_rate=inner_lr)
    for i in range(inner_steps):
        with tf.GradientTape() as tape:
            predictions = model_copy(support_data)
            loss = dice_loss(support_labels, predictions)
        gradients = tape.gradient(loss, model_copy.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model_copy.trainable_variables))
        print(f"Inner Step: {i+1}, Loss: {loss.numpy()}")
    return model_copy

def evaluate_adapted_model(model, query_data, query_labels):
    predictions = model(query_data)
    loss = dice_loss(query_labels, predictions)
    return loss.numpy()

In [ ]:
locations_meta_testing = ['Covington']
# locations_meta_testing = ['Rowancreek']
# locations_meta_training = ['Alexander', 'Rowancreek']
meta_test_episodes = data_loader.create_multi_episodes(4, locations_meta_testing)


Covington
Covington
Covington
Covington


In [ ]:
name = "maml_1_500_1_20240503_201907_best_model"
base_model = tf.keras.models.load_model('models/'+name)

# Example usage:
for test_episode in meta_test_episodes:
    adapted_model = adapt_to_new_task(base_model, test_episode["support_set_data"], test_episode["support_set_labels"], inner_lr=0.001, inner_steps=400)
    test_loss = evaluate_adapted_model(adapted_model, test_episode["query_set_data"], test_episode["query_set_labels"])
    print(f"Test Loss after adaptation: {test_loss}")

adapted_model.save('models/'+name+'/adapted_covington')  # Save the best model

Inner Step: 1, Loss: 0.7070581912994385
Inner Step: 2, Loss: 0.6860436201095581
Inner Step: 3, Loss: 0.648269772529602
Inner Step: 4, Loss: 0.616706132888794
Inner Step: 5, Loss: 0.6115401983261108
Inner Step: 6, Loss: 0.6031354665756226
Inner Step: 7, Loss: 0.588828980922699
Inner Step: 8, Loss: 0.576479434967041
Inner Step: 9, Loss: 0.5720885992050171
Inner Step: 10, Loss: 0.5728148221969604
Inner Step: 11, Loss: 0.5644044876098633
Inner Step: 12, Loss: 0.555587887763977
Inner Step: 13, Loss: 0.5521668195724487
Inner Step: 14, Loss: 0.5511040687561035
Inner Step: 15, Loss: 0.5485281944274902
Inner Step: 16, Loss: 0.5442174077033997
Inner Step: 17, Loss: 0.5417673587799072
Inner Step: 18, Loss: 0.5420837998390198
Inner Step: 19, Loss: 0.5413864850997925
Inner Step: 20, Loss: 0.5384175777435303
Inner Step: 21, Loss: 0.536340594291687
Inner Step: 22, Loss: 0.5360579490661621
Inner Step: 23, Loss: 0.5344541072845459
Inner Step: 24, Loss: 0.5314226746559143
Inner Step: 25, Loss: 0.5293649

In [ ]:
%cd '/content/drive/MyDrive/Meta_learning_research/Notebooks/'
input_data = './samples/'
location = "Covinton"

X_test = np.load('/content/drive/MyDrive/Meta_learning_research/Notebooks/samples/Covington/bottom_half_test_data.npy').astype(np.float32)
X_test[X_test < 0] = 0

/content/drive/MyDrive/Meta_learning_research/Notebooks


In [ ]:
# This normalization_type was define on the top of the notebook for the dataloader
if normalization_type == '0':
    data_min = 0
    data_max = 255
    X_test_norm = (X_test - data_min) / (data_max - data_min)
elif normalization_type == '-1':
    data_min = 0
    data_max = 255
    X_test_norm = 2 * ((X_test - data_min) / (data_max - data_min)) - 1
elif normalization_type == 'none':
    X_test_norm = X_test
else:
    raise ValueError("Unsupported normalization type. Choose '0-1' or '-1-1'.")

In [ ]:
prediction = adapted_model.predict(X_test_norm)

np.save('/content/drive/MyDrive/Meta_learning_research/Notebooks/predicts/'+name+".npy", prediction)
print('/content/drive/MyDrive/Meta_learning_research/Notebooks/predicts/'+name+".npy")

115/115 [==============================] - 5s 30ms/step
/content/drive/MyDrive/Meta_learning_research/Notebooks/predicts/maml_1_500_1_20240503_201907_best_model.npy


In [ ]:
print('/content/drive/MyDrive/Meta_learning_research/Notebooks/predicts/'+name+".npy")

/content/drive/MyDrive/Meta_learning_research/Notebooks/predicts/maml_1_500_1_20240430_013450_best_model.npy


# Visualization for the input data

In [ ]:
def visualize_images(image_stack, labels, num_images=25):
    """
    Visualizes images with their respective channels and labels.

    Args:
    - image_stack (numpy.ndarray): An array of shape (N, 224, 224, 8) where N is the number of images.
    - labels (numpy.ndarray): An array of labels of shape (N, 224, 224).
    - num_images (int): Number of images to visualize.
    """
    fig, axs = plt.subplots(nrows=num_images, ncols=9, figsize=(18, 2 * num_images))

    for i in range(min(num_images, image_stack.shape[0])):
        # Normalize each channel for visualization purposes
        for ch in range(8):
            axs[i, ch].imshow(image_stack[i, :, :, ch], cmap='gray', aspect='auto')
            axs[i, ch].axis('off')  # Turn off axis
        # Adding the label image in the last column
        axs[i, 8].imshow(labels[i], cmap='gray', aspect='auto')
        axs[i, 8].axis('off')

    plt.subplots_adjust(wspace=0.05, hspace=0.05)
    plt.show()

In [ ]:
eps_idx= 0

visualize_images(meta_test_episodes[eps_idx]["support_set_data"], meta_test_episodes[eps_idx]["support_set_labels"], num_images=25)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
eps_idx= 0

visualize_images(meta_test_episodes[eps_idx]["query_set_data"], meta_test_episodes[eps_idx]["query_set_labels"], num_images=25)